In [1]:
# imports

import pandas as pd
import os
import requests 
import json
from IPython.display import display
import math
import sqlite3

In [2]:
#set the keys

foursquare_api = os.environ['FOURSQUARE_API_KEY']
api_key = os.environ['YELP_API_KEY']
google_key = os.environ['GOOGLE_APP_KEY']

In [3]:
# Foursquest API client setup

base = "https://api.foursquare.com/v3"
url = "/places/search"
ll = '49.2856,-123.1115'   # Waterfront Station, Vancouver, BC
lat = '49.2856'
long = '-123.1115'
radius = '1000'
categories = '13065'   # Restaurants
params = f'?ll={ll}&radius={radius}&categories={categories}&limit=50&&fields=rating%2Cname%2Cdistance%2Clocation%2Cstats'

headers = {'accept': 'application/json', 'Authorization': foursquare_api}

# Foursquare

Send a request to Foursquare with a small radius (1000m) with the location of your choice

In [4]:
# Safeguard for invalid response

try:
    response = requests.get(base+url+params, headers = headers).json()
    response['results']   
    display(response['results'])
except:     
    print('Something went wrong :(')

[{'distance': 129,
  'location': {'address': '200 Granville St',
   'address_extended': '# 70',
   'country': 'CA',
   'cross_street': 'West Cordova Street',
   'formatted_address': '200 Granville St (West Cordova Street), Vancouver BC V6C 1S4',
   'locality': 'Vancouver',
   'neighborhood': ['Downtown'],
   'postcode': 'V6C 1S4',
   'region': 'BC'},
  'name': 'Miku',
  'rating': 9.0,
  'stats': {'total_photos': 950, 'total_ratings': 609, 'total_tips': 178}},
 {'distance': 54,
  'location': {'address': '375 Water St',
   'address_extended': '# 109',
   'country': 'CA',
   'cross_street': 'at Richards St',
   'formatted_address': '375 Water St (at Richards St), Vancouver BC V6B 5C6',
   'locality': 'Vancouver',
   'neighborhood': ['Gastown'],
   'postcode': 'V6B 5C6',
   'region': 'BC'},
  'name': 'Steamworks Brewing Co',
  'rating': 8.4,
  'stats': {'total_photos': 1114, 'total_ratings': 907, 'total_tips': 182}},
 {'distance': 312,
  'location': {'address': '325 Cambie St',
   'country

Parse through the response to get the POI details you want (rating, name, location, etc) AND Put your parsed results into a DataFrame

In [5]:
# Create DataFrame from JSON
# Check for valid results, filter and sort POI


if 'results' in response:
    foursquare_raiting = pd.json_normalize(response['results'])
    results_table = foursquare_raiting.filter(items=['rating', 'name', 'location.address', 'distance', 'stats.total_ratings']).sort_values(by=['rating', 'stats.total_ratings'], ascending = False).rename(columns={'location.address': 'address', 'stats.total_ratings': 'review_count', 'distance': 'distance_in_meters'})
    display(results_table)
else:
    print('Error in fatching data')

,rating,name,address,distance_in_meters,review_count
2,9.3,Revolver,325 Cambie St,312,795
3,9.3,FlyOver Canada,999 Canada Pl Way,397,369
5,9.2,Gotham Steakhouse & Cocktail Bar,615 Seymour St,441,172
17,9.1,Birds & the Beets,55 Powell St,660,163
0,9.0,Miku,200 Granville St,129,609
29,9.0,Joe Fortes Seafood & Chop House,777 Thurlow St,938,529
15,9.0,Hawksworth Restaurant,801 Georgia St W,628,270
32,9.0,Keefer Bar,135 Keefer St,989,229
11,9.0,MeeT in Gastown,12 Water St,545,174
22,8.9,Cafe Medina,780 Richards St,681,1110


# Yelp

Send a request with the same location paramaters (location, radius, etc) AND Parse through your result and get POI details

In [6]:
# Import Yelp package and setup Yelp API client 


from yelpapi import YelpAPI
with YelpAPI(api_key) as yelp_api:
    yelp_business_data = yelp_api.search_query(latitude=lat, longitude=long, radius = '1000', categories = 'restaurants')

    
if 'businesses' in yelp_business_data:
    yelp_business_data = yelp_business_data['businesses']

In [7]:
# Check yelp data
    
yelp_business_data

[{'id': '4EV_ZcQmjAmP3pmO-_nb2A',
  'alias': 'miku-vancouver-2',
  'name': 'Miku',
  'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/sXiX56IK_jTBX742_KKYlw/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/miku-vancouver-2?adjust_creative=0knJKTyxri8wdedBsuzw5A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=0knJKTyxri8wdedBsuzw5A',
  'review_count': 2019,
  'categories': [{'alias': 'japanese', 'title': 'Japanese'},
   {'alias': 'sushi', 'title': 'Sushi Bars'}],
  'rating': 4.5,
  'coordinates': {'latitude': 49.28691753455261,
   'longitude': -123.11276314110472},
  'transactions': [],
  'price': '$$$',
  'location': {'address1': '200 Granville Street',
   'address2': 'Suite 70',
   'address3': '',
   'city': 'Vancouver',
   'zip_code': 'V6C 1S4',
   'country': 'CA',
   'state': 'BC',
   'display_address': ['200 Granville Street',
    'Suite 70',
    'Vancouver, BC V6C 1S4',
    'Canada']},
  'phone': '+16045683900',
  'display_phone': '+1 604

Put your parsed results into a DataFrame

In [8]:
if len(yelp_business_data):

    for item in yelp_business_data:
        item['distance'] = math.ceil(item['distance'])
        
    yelp_business_data_result = pd.json_normalize(yelp_business_data)
    yelp_results_table = yelp_business_data_result.filter(items=['rating', 'name', 'location.address1', 'distance', 'review_count']).sort_values(by=['rating', 'review_count'], ascending = False).rename(columns={'location.address1': 'address', 'distance':'distance_in_meters'})
    display(yelp_results_table)
else:
    print('No businessess were found')

,rating,name,address,distance_in_meters,review_count
0,4.5,Miku,200 Granville Street,173,2019
2,4.5,Jam Cafe on Beatty,556 Beatty Street,610,1246
10,4.5,Fanny Bay Oyster Bar & Shellfish Market,762 Cambie St,801,667
6,4.5,La Taqueria Pinche Taco Shop,322 W Hastings Street,307,501
14,4.5,The Poke Guy,420 Richards Street,208,201
1,4.0,Medina Cafe,780 Richards Street,701,2479
3,4.0,Chambar,568 Beatty Street,618,1479
19,4.0,Phnom Penh,244 E Georgia Street,1252,1478
5,4.0,Twisted Fork,213 Carral Street,578,1077
18,4.0,Thierry,1059 Alberni Street,817,895


# Google (stretch)

Use the same process as the first two APIs

In [9]:

 # "https://maps.googleapis.com/maps/api/place/details/json?place_id=ChIJN1t_tDeuEmsRUsoyG83frY4&fields=name%2Crating%2Cformatted_phone_number&key=GOOGLE_APP_KEY"

url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={ll}&radius={radius}&type=restaurant&key={google_key}'

headers = {}
response = requests.request("GET", url, headers=headers).json()
response        

{'html_attributions': [],
 'next_page_token': 'AcYSjRjAuMw-acJ-zIX6nLr4DN6Y8oabzZyf0Jc5Y9hOz8-NH8vrkjb6bSOas1szFONSNMNjXhdGp_50Ngydp1mRer3opR5--n8Xs5Nu91W_7p8-JlkW2xPcS9Ls1ROldKd5okutpMfvf0IjbNuw9-bbQ7RxnWW4ao0GSR3luwmaPhcBzwzg3Z6fjcCybM4sOGhUGW6kehGL1sEnVLsQOCjR7zxOl19DFHaS29ePE8cSGcqCmywgF6JraAWX-x2s5XF6PgrHcH2Y0AJwV2Fbm0WcPFAF616BdsClnAFbWIa9dvoAZOH3nTN5knLOPcBGo2pfF1glcU0Kwbcux9XMQA8W_pXAJHHew0M_lkBoaMbf-atVPgmF0LCcF0lmvpa2voL_5g06laGEWdCSXsTbfSB89dXFWppavVUMctrDPTB8JjwjSzJPS5yBTvVL8KHj',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 49.2837112, 'lng': -123.1211176},
    'viewport': {'northeast': {'lat': 49.2850760302915,
      'lng': -123.1196651197085},
     'southwest': {'lat': 49.2823780697085, 'lng': -123.1223630802915}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'icon_background_color': '#909CE1',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet'

In [10]:
google_data_id = []
if len(response['results']):
    for record in response['results']:
        if 'place_id' in record:
            place_id = record['place_id']
            
        if place_id :
            url_id = f'https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&fields=name%2Crating%2Cuser_ratings_total%2Cformatted_address&key={google_key}'
            response_id = requests.get(url_id)
        if response_id.status_code == 200:
            google_data_id.append({'place_id': place_id, **response_id.json()})

        
google_data_id      

[{'place_id': 'ChIJHbH93TGoqEwRMwrQGCOk9B4',
  'html_attributions': [],
  'result': {'formatted_address': '900 W Georgia St, Vancouver, BC V6C 2W6, Canada',
   'name': 'Fairmont Hotel Vancouver',
   'rating': 4.5,
   'user_ratings_total': 4645},
  'status': 'OK'},
 {'place_id': 'ChIJgzLyCINxhlQRkHq8urzYQ2Y',
  'html_attributions': [],
  'result': {'formatted_address': '1038 Canada Pl, Vancouver, BC V6C 0B9, Canada',
   'name': 'Fairmont Pacific Rim',
   'rating': 4.6,
   'user_ratings_total': 4254},
  'status': 'OK'},
 {'place_id': 'ChIJYyXi7YNxhlQRUxgGSBaVYGc',
  'html_attributions': [],
  'result': {'formatted_address': '1133 W Hastings St, Vancouver, BC V6E 3T3, Canada',
   'name': 'Pinnacle Hotel Harbourfront',
   'rating': 4.1,
   'user_ratings_total': 3627},
  'status': 'OK'},
 {'place_id': 'ChIJq2tI6HtxhlQRBaKUv59cS9E',
  'html_attributions': [],
  'result': {'formatted_address': '568 Beatty St, Vancouver, BC V6B 2L3, Canada',
   'name': 'Chambar Restaurant',
   'rating': 4.5,
 

In [11]:

google_restaurants = pd.json_normalize(google_data_id)
google_results = google_restaurants.filter(items=['place_id', 'result.rating', 'result.name', 'result.formatted_address', 'result.user_ratings_total']).sort_values(by=['place_id'], ascending = False)
google_results

,place_id,result.rating,result.name,result.formatted_address,result.user_ratings_total
12,ChIJxRdotIFxhlQRY7DFe7zRkho,4.2,Italian Kitchen,"860 Burrard St, Vancouver, BC V6Z 1X9, Canada",2006
13,ChIJx19lhHhxhlQR00Wyx2MBMZc,4.2,Trees Organic Coffee,"450 Granville St, Vancouver, BC V6C 1V4, Canada",1479
7,ChIJwzHipIFxhlQRSKY9IKkeNks,3.3,Subway,"1055 W Georgia St Unit 232, Vancouver, BC V6E ...",12
10,ChIJv9OoAYBxhlQRBV7HNkJ_R9c,4.2,JAPADOG,"530 Robson St, Vancouver, BC V6B 2B7, Canada",3349
6,ChIJqXfGF3xxhlQRAvL-tqjuoZk,3.6,Moxie's Grill & Bar,"180 W Georgia St, Vancouver, BC V6B 4P4, Canada",398
3,ChIJq2tI6HtxhlQRBaKUv59cS9E,4.5,Chambar Restaurant,"568 Beatty St, Vancouver, BC V6B 2L3, Canada",3347
5,ChIJi9Ix4HlxhlQRR9d6rVYjsFA,4.4,Jules_Bar à Vin_,"216 Abbott St, Vancouver, BC V6B 2K8, Canada",621
1,ChIJgzLyCINxhlQRkHq8urzYQ2Y,4.6,Fairmont Pacific Rim,"1038 Canada Pl, Vancouver, BC V6C 0B9, Canada",4254
15,ChIJfe2sYHhxhlQR6YR9clFor1Y,3.4,Tim Hortons,"555 W Hastings St Unit 6, Vancouver, BC V6B 5K...",394
2,ChIJYyXi7YNxhlQRUxgGSBaVYGc,4.1,Pinnacle Hotel Harbourfront,"1133 W Hastings St, Vancouver, BC V6E 3T3, Canada",3627


In [12]:
response_distance = []

if len(response['results']):
    for record in response['results']:
        if 'place_id' in record:
            place_id = record['place_id']
            
        if place_id:
            url_distance = f'https://maps.googleapis.com/maps/api/distancematrix/json?origins={ll}&destinations=place_id:{place_id}&units=imperial&key={google_key}'

            rp = requests.get(url_distance)
        if rp.status_code == 200:
            response_distance.append({ 'place_id': place_id, **rp.json()})
        

In [13]:

def get_meters(miles):
    conversion_factor = 0.62137119
    
    meters = (float(miles) / conversion_factor) * 1000
    return math.ceil(meters)

result = []
for item in response_distance:
    miles = item['rows'][0]['elements'][0]['distance']['text']

    meters = get_meters(miles.split(' ')[0])
    
    result.append({ **item, 'distance_in_meters': meters })

google_distance = pd.json_normalize(result)
# display(google_distance)
google_distances_table = google_distance.filter(items=['place_id', 'distance_in_meters']).sort_values(by=['place_id'], ascending = False)
google_distances_table

,place_id,distance_in_meters
12,ChIJxRdotIFxhlQRY7DFe7zRkho,1127
13,ChIJx19lhHhxhlQR00Wyx2MBMZc,483
7,ChIJwzHipIFxhlQRSKY9IKkeNks,966
10,ChIJv9OoAYBxhlQRBV7HNkJ_R9c,805
6,ChIJqXfGF3xxhlQRAvL-tqjuoZk,805
3,ChIJq2tI6HtxhlQRBaKUv59cS9E,805
5,ChIJi9Ix4HlxhlQRR9d6rVYjsFA,1288
1,ChIJgzLyCINxhlQRkHq8urzYQ2Y,483
15,ChIJfe2sYHhxhlQR6YR9clFor1Y,322
2,ChIJYyXi7YNxhlQRUxgGSBaVYGc,805


In [14]:
google_df = pd.merge(google_results, google_distances_table)
google_df.sort_values(by=['result.rating'], ascending = False)

final_google_df = google_df.iloc[:, 1:].sort_values(by=['result.rating','result.user_ratings_total'], ascending = False).rename(columns={'result.rating': 'rating', 'result.name': 'name', 'result.formatted_address': 'address', 'result.user_ratings_total': 'review_count'})


columns_titles = ['rating', 'name', 'address', 'distance_in_meters', 'review_count']
final_google_table=final_google_df.reindex(columns=columns_titles)
final_google_table

,rating,name,address,distance_in_meters,review_count
7,4.6,Fairmont Pacific Rim,"1038 Canada Pl, Vancouver, BC V6C 0B9, Canada",483,4254
10,4.6,Gotham Steakhouse & Cocktail Bar,"615 Seymour St, Vancouver, BC V6B 3K3, Canada",805,2013
12,4.5,Fairmont Hotel Vancouver,"900 W Georgia St, Vancouver, BC V6C 2W6, Canada",966,4645
5,4.5,Chambar Restaurant,"568 Beatty St, Vancouver, BC V6B 2L3, Canada",805,3347
13,4.5,Terminal City Club,"837 W Hastings St, Vancouver, BC V6C 1B6, Canada",322,508
14,4.4,The Old Spaghetti Factory (Gastown),"53 Water St, Vancouver, BC V6B 1A1, Canada",1127,6202
16,4.4,Cactus Club Cafe Coal Harbour,"1085 Canada Pl, Vancouver, BC V6C 3E1, Canada",644,4792
6,4.4,Jules_Bar à Vin_,"216 Abbott St, Vancouver, BC V6B 2K8, Canada",1288,621
11,4.3,Cactus Club Cafe Bentall 5,"588 Burrard St, Vancouver, BC V6C 0A8, Canada",644,2349
19,4.3,Catch 122 Cafe Bistro,"120 W Hastings St, Vancouver, BC V6B 1G8, Canada",483,1291


# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [15]:
## 3 DataFrames:
# 1. results_table     - foursquare
# 2. yelp_results_table   - yelp
# 3. final_google_df   - google


display(results_table)
display(yelp_results_table)
display(final_google_table)


,rating,name,address,distance_in_meters,review_count
2,9.3,Revolver,325 Cambie St,312,795
3,9.3,FlyOver Canada,999 Canada Pl Way,397,369
5,9.2,Gotham Steakhouse & Cocktail Bar,615 Seymour St,441,172
17,9.1,Birds & the Beets,55 Powell St,660,163
0,9.0,Miku,200 Granville St,129,609
29,9.0,Joe Fortes Seafood & Chop House,777 Thurlow St,938,529
15,9.0,Hawksworth Restaurant,801 Georgia St W,628,270
32,9.0,Keefer Bar,135 Keefer St,989,229
11,9.0,MeeT in Gastown,12 Water St,545,174
22,8.9,Cafe Medina,780 Richards St,681,1110


,rating,name,address,distance_in_meters,review_count
0,4.5,Miku,200 Granville Street,173,2019
2,4.5,Jam Cafe on Beatty,556 Beatty Street,610,1246
10,4.5,Fanny Bay Oyster Bar & Shellfish Market,762 Cambie St,801,667
6,4.5,La Taqueria Pinche Taco Shop,322 W Hastings Street,307,501
14,4.5,The Poke Guy,420 Richards Street,208,201
1,4.0,Medina Cafe,780 Richards Street,701,2479
3,4.0,Chambar,568 Beatty Street,618,1479
19,4.0,Phnom Penh,244 E Georgia Street,1252,1478
5,4.0,Twisted Fork,213 Carral Street,578,1077
18,4.0,Thierry,1059 Alberni Street,817,895


,rating,name,address,distance_in_meters,review_count
7,4.6,Fairmont Pacific Rim,"1038 Canada Pl, Vancouver, BC V6C 0B9, Canada",483,4254
10,4.6,Gotham Steakhouse & Cocktail Bar,"615 Seymour St, Vancouver, BC V6B 3K3, Canada",805,2013
12,4.5,Fairmont Hotel Vancouver,"900 W Georgia St, Vancouver, BC V6C 2W6, Canada",966,4645
5,4.5,Chambar Restaurant,"568 Beatty St, Vancouver, BC V6B 2L3, Canada",805,3347
13,4.5,Terminal City Club,"837 W Hastings St, Vancouver, BC V6C 1B6, Canada",322,508
14,4.4,The Old Spaghetti Factory (Gastown),"53 Water St, Vancouver, BC V6B 1A1, Canada",1127,6202
16,4.4,Cactus Club Cafe Coal Harbour,"1085 Canada Pl, Vancouver, BC V6C 3E1, Canada",644,4792
6,4.4,Jules_Bar à Vin_,"216 Abbott St, Vancouver, BC V6B 2K8, Canada",1288,621
11,4.3,Cactus Club Cafe Bentall 5,"588 Burrard St, Vancouver, BC V6C 0A8, Canada",644,2349
19,4.3,Catch 122 Cafe Bistro,"120 W Hastings St, Vancouver, BC V6B 1G8, Canada",483,1291


In [16]:
# DB connect

conn = sqlite3.connect('../data/restaurants_ratings.db')
c = conn.cursor()

In [17]:
# Creat FourSquare Database

c.execute('CREATE TABLE IF NOT EXISTS foursquare_raiting (rating, name, address, distance_in_meters, review_count)')
conn.commit()
results_table.to_sql('foursquare_raiting', conn, if_exists='replace', index = False)

# Creat Yelp Database

c.execute('CREATE TABLE IF NOT EXISTS yelp_raiting (rating, name, address, distance_in_meters, review_count)')
conn.commit()
yelp_results_table.to_sql('yelp_raiting', conn, if_exists='replace', index = False)

# Create Google Database

c.execute('CREATE TABLE IF NOT EXISTS google_raiting (rating, name, address, distance_in_meters, review_count)')
conn.commit()
final_google_table.to_sql('google_raiting', conn, if_exists='replace', index = False)

20

Get the top 10 restaurants according to their rating

In [18]:
# In order to make data more representative of true rating we are sorting by rating and numbers of reviews

c.execute('SELECT * FROM foursquare_raiting ORDER BY rating desc, review_count desc limit 10') 

foursquare_df = pd.DataFrame(c.fetchall(), columns = ['rating', 'name', 'address', 'distance in meters', 'review count'])
display(foursquare_df)

,rating,name,address,distance in meters,review count
0,9.3,Revolver,325 Cambie St,312,795
1,9.3,FlyOver Canada,999 Canada Pl Way,397,369
2,9.2,Gotham Steakhouse & Cocktail Bar,615 Seymour St,441,172
3,9.1,Birds & the Beets,55 Powell St,660,163
4,9.0,Miku,200 Granville St,129,609
5,9.0,Joe Fortes Seafood & Chop House,777 Thurlow St,938,529
6,9.0,Hawksworth Restaurant,801 Georgia St W,628,270
7,9.0,Keefer Bar,135 Keefer St,989,229
8,9.0,MeeT in Gastown,12 Water St,545,174
9,8.9,Cafe Medina,780 Richards St,681,1110


In [19]:
# In order to make data more representative of true rating we are sorting by rating and numbers of reviews

c.execute('SELECT * FROM yelp_raiting ORDER BY rating desc, review_count desc limit 10') 

yelp_df = pd.DataFrame(c.fetchall(), columns = ['rating', 'name', 'address', 'distance in meters', 'review count'])
display(yelp_df)

,rating,name,address,distance in meters,review count
0,4.5,Miku,200 Granville Street,173,2019
1,4.5,Jam Cafe on Beatty,556 Beatty Street,610,1246
2,4.5,Fanny Bay Oyster Bar & Shellfish Market,762 Cambie St,801,667
3,4.5,La Taqueria Pinche Taco Shop,322 W Hastings Street,307,501
4,4.5,The Poke Guy,420 Richards Street,208,201
5,4.0,Medina Cafe,780 Richards Street,701,2479
6,4.0,Chambar,568 Beatty Street,618,1479
7,4.0,Phnom Penh,244 E Georgia Street,1252,1478
8,4.0,Twisted Fork,213 Carral Street,578,1077
9,4.0,Thierry,1059 Alberni Street,817,895


In [20]:
# In order to make data more representative of true rating we are sorting by rating and numbers of reviews

c.execute('SELECT * FROM google_raiting ORDER BY rating desc, review_count desc limit 10') 

google_df = pd.DataFrame(c.fetchall(), columns = ['rating', 'name', 'address', 'distance in meters', 'review count'])
display(google_df)

,rating,name,address,distance in meters,review count
0,4.6,Fairmont Pacific Rim,"1038 Canada Pl, Vancouver, BC V6C 0B9, Canada",483,4254
1,4.6,Gotham Steakhouse & Cocktail Bar,"615 Seymour St, Vancouver, BC V6B 3K3, Canada",805,2013
2,4.5,Fairmont Hotel Vancouver,"900 W Georgia St, Vancouver, BC V6C 2W6, Canada",966,4645
3,4.5,Chambar Restaurant,"568 Beatty St, Vancouver, BC V6B 2L3, Canada",805,3347
4,4.5,Terminal City Club,"837 W Hastings St, Vancouver, BC V6C 1B6, Canada",322,508
5,4.4,The Old Spaghetti Factory (Gastown),"53 Water St, Vancouver, BC V6B 1A1, Canada",1127,6202
6,4.4,Cactus Club Cafe Coal Harbour,"1085 Canada Pl, Vancouver, BC V6C 3E1, Canada",644,4792
7,4.4,Jules_Bar à Vin_,"216 Abbott St, Vancouver, BC V6B 2K8, Canada",1288,621
8,4.3,Cactus Club Cafe Bentall 5,"588 Burrard St, Vancouver, BC V6C 0A8, Canada",644,2349
9,4.3,Catch 122 Cafe Bistro,"120 W Hastings St, Vancouver, BC V6B 1G8, Canada",483,1291


# Travelling Salesman Problem (stretch)

If you have time, follow the steps in the [ortools tutorial](https://developers.google.com/optimization/routing/tsp) using Google's [Directions API](https://developers.google.com/maps/documentation/directions/start).